In [9]:
import numpy as np
import mediapipe as mp
import os
import pandas as pd

In [10]:
def process_raw_features(raw_features: mp.tasks.vision.HandLandmarkerResult):
    landmarks = []
    for hand_landmarks in results.hand_landmarks:
        x_coordinates = np.array([landmark.x for landmark in hand_landmarks])
        x_min, x_max = x_coordinates.min(), x_coordinates.max()
        norm_x_coordinates = (x_coordinates - x_min) / (x_max - x_min)
        y_coordinates = np.array([landmark.y for landmark in hand_landmarks])
        y_min, y_max = y_coordinates.min(), y_coordinates.max()
        norm_y_coordinates = (y_coordinates - y_min) / (y_max - y_min)
        z_coordinates = np.array([landmark.z for landmark in hand_landmarks])

        landmarks.append(np.array([
            [x, y, z] for x, y, z in zip(norm_x_coordinates, norm_y_coordinates,     z_coordinates)
        ]).flatten())
    return landmarks

In [11]:
columns = []
for i in range(21):  # 21 hand landmarks
    columns += [f'x{i}', f'y{i}', f'z{i}']
columns += ["Class"]
df = pd.DataFrame(columns=columns)
df

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,x18,y18,z18,x19,y19,z19,x20,y20,z20,Class


In [12]:
# Initialize MediaPipe HandLandmarker
base_options = mp.tasks.BaseOptions(model_asset_path='hand_landmarker.task')
options = mp.tasks.vision.HandLandmarkerOptions(
    base_options=base_options, num_hands=1, running_mode=mp.tasks.vision.RunningMode.IMAGE
)

# Folder containing your images
image_folder = 'new_images/'
# Loop through all images
with mp.tasks.vision.HandLandmarker.create_from_options(options) as mp_hands:
    for cls in os.listdir(image_folder):
        cls_dir = os.path.join(image_folder, cls)
        for filename in os.listdir(cls_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(cls_dir, filename)
                image = mp.Image.create_from_file(image_path)
                results = mp_hands.detect(image)
                landmarks_np = processed_values = process_raw_features(results)
                row = list(landmarks_np[0]) + [cls]
                # print (row, len(row))
                df.loc[len(df)] = row


In [14]:
df.to_csv('hand_landmarks_with_class.csv', index=False)